In [1]:
%run /home/bb/src/python/valkyrie/apps/common/run_notebook_import.py

/media/veracrypt1/python/valkyrie/apps/common/run_notebook_import.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/media/veracrypt1/python/valkyrie/lib/valkyrie/ml/data.py:291: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [3]:
import copy
from itertools import product as cartesian_product

from overrides import overrides
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.nn import functional as F

from valkyrie.tools import *
from valkyrie.ml import modules
from valkyrie.ml.utils import tensor, HyperParameters
from valkyrie.ml import utils as ml_utils
from valkyrie.ml.net import Regressor, BatchNormAlongW
from valkyrie.nibelungen.data import DataMgr
from valkyrie.ml.data import Df2T2, SubDataSet

In [4]:
from valkyrie.ml.data import DataModule

sdate, edate = '2023-01-01', '2023-06-30'
instrs = ['BDM.BTC.USDT.FP']
freq = '1s'
ret_n_s = [120]
root_dir = '/home/bb/data/BDM'
#########################################################
#DataMgr
#########################################################
data_mgr = DataMgr(sdate, edate, freq, root_dir, instrs = instrs, ret_n_s = ret_n_s)
df_res = data_mgr.get(instrs[0]) 
channels = ['bpx_last','apx_last'] + ['bq_last','aq_last','buy_qty_sum', 'sell_qty_sum'] 
n_channels = len(channels)
H, W = 16, 64
df2t2 = Df2T2(df_res, H, W, xcols = channels, ycol = 'mid_last_ret_120_n', yscaler = 1)

train_set, val_set = SubDataSet.create_train_valiation_pair(df2t2, 0.8)
dm = DataModule.from_dataset(train_set, val_set, 2048)

after dropping na 15638095 -> 15605328


# TEST NET

In [ ]:
ne = NetEvaluator(testNet, dm)
ne.eval_performance()

# GoogleNet

In [ ]:
class Inception(nn.Module):
    # c1--c4 are the number of output channels for each branch
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Branch 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        # Branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)

    def forward(self, x):
        b1 = F.leaky_relu(self.b1_1(x), 0.1)
        b2 = F.leaky_relu(self.b2_2(F.leaky_relu(self.b2_1(x), 0.1)), 0.1)
        b3 = F.leaky_relu(self.b3_2(F.leaky_relu(self.b3_1(x), 0.1)), 0.1)
        b4 = F.leaky_relu(self.b4_2(self.b4_1(x)), 0.1)
        return torch.cat((b1, b2, b3, b4), dim=1)
        
class GoogleNet(Regressor):
    def b1(self):
        return nn.Sequential(            
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LeakyReLU(0.1), nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    

    def b2(self):
        return nn.Sequential(        
            nn.LazyConv2d(64, kernel_size=1), nn.LeakyReLU(0.1),
            nn.LazyConv2d(192, kernel_size=3, padding=1), nn.LeakyReLU(0.1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))


    def b3(self):
        return nn.Sequential(        
            Inception(64, (96, 128), (16, 32), 32),
            Inception(128, (128, 192), (32, 96), 64),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))


    def b4(self):
        return nn.Sequential(
                             Inception(192, (96, 208), (16, 48), 64),
                             Inception(160, (112, 224), (24, 64), 64),
                             Inception(128, (128, 256), (24, 64), 64),
                             Inception(112, (144, 288), (32, 64), 64),
                             Inception(256, (160, 320), (32, 128), 128),
                             nn.MaxPool2d(kernel_size=3, stride=2, padding=1))


    def b4(self):
        return nn.Sequential(
                             Inception(192, (96, 208), (16, 48), 64),
                             Inception(160, (112, 224), (24, 64), 64),
                             Inception(128, (128, 256), (24, 64), 64),
                             Inception(112, (144, 288), (32, 64), 64),
                             Inception(256, (160, 320), (32, 128), 128),
                             nn.MaxPool2d(kernel_size=3, stride=2, padding=1))


    def b5(self):
        return nn.Sequential(
                             Inception(256, (160, 320), (32, 128), 128),
                             Inception(384, (192, 384), (48, 128), 128),
                             nn.AdaptiveAvgPool2d((1,1)), nn.Flatten())


    def b0(self):
        return BatchNormAlongW(self.n_channel, self.height)

    def __init__(self, n_channel, height, lr=0.1, num_classes=10):
        #super(GoogleNet, self).__init__()
        self.n_channel = n_channel
        self.height = height
        super(GoogleNet, self).__init__('l1')        
        self.save_hyperparameters()
        self.net = nn.Sequential(self.b0(), self.b1(), self.b2(), self.b3(), self.b4(),
                                 self.b5(), nn.LazyLinear(1))
        self.net.apply(d2l.init_cnn)

In [ ]:
google_net = GoogleNet(n_channels, H, lr = 0.0001)
X = df2t2[0][0]
ml_utils.layer_summary(google_net, X.shape)

In [ ]:
trainer = modules.Trainer(max_epochs=200, num_gpus=1)
with Timer('training'):
    trainer.fit(google_net, dm)

In [ ]:
ne = NetEvaluator(google_net, dm)
ne.eval_performance()

In [ ]:
class MyCNN(nn.Module):
    def __init__(self,num_channels = n_channels, height = 4):
        super(MyCNN, self).__init__()        
        self.bn_along_w = BatchNormAlongW(num_channels, height)  # Example dimensions
        # ... other layers ...

    def forward(self, x):        
        x = self.bn_along_w(x)
        # ... operations for other layers ...
        return x

# Instantiate the model
model = MyCNN()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 3  # Number of training epochs

for epoch in range(num_epochs):
    for inputs, labels in dm.get_dataloader(True):
        inputs = inputs
        labels = torch.randn_like(inputs)
        optimizer.zero_grad()  # Zero the gradient buffers
        outputs = model(inputs)        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [ ]:
outputs.shape

In [ ]:
inputs.shape

In [ ]:
0.0381 * np.sqrt(1499)

In [ ]:
inputs.mean(axis=0)

In [ ]:
outputs.mean(axis=0)

In [ ]:
pd.Series({'a':2,'b':3})